In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

/Users/rmizuta/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/japanize_matplotlib/__init__.py:13: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = font_manager.createFontList(font_files)


In [2]:
users = pd.read_csv('../ml-1m/users.dat', sep="::", engine = 'python',names=["userID","gender","age","occupation","zipcpde"])
data_frame = pd.read_csv('../ml-1m/ratings.dat', sep="::", usecols = [0, 1, 2], names = ['userID', 'movieID', 'rating'], engine = 'python')
data_mat=pd.pivot_table(data_frame,index = 'userID', columns = 'movieID', values = 'rating').fillna(0).reset_index(drop=True)

In [3]:
data_mat.head()

movieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#集計用テーブル作成
cluster_feat=users.copy()
cluster_feat['age']=cluster_feat['age'].astype(str)
cluster_feat=pd.concat([cluster_feat,pd.get_dummies(cluster_feat[['gender','age']])],axis=1)
del cluster_feat['gender']
del cluster_feat['age']
del cluster_feat['occupation']
del cluster_feat['zipcpde']

In [5]:
#時間計測用
from functools import wraps
import time
def stop_watch(func) :
    @wraps(func)
    def wrapper(*args, **kargs) :
        start = time.time()
        result = func(*args,**kargs)
        process_time =  time.time() - start
        print(f"{func.__name__}は{process_time}秒かかりました")
        return result
    return wrapper

In [6]:
from sklearn.cluster import KMeans
@stop_watch
def do_kmeans(df):
    kmeans = KMeans(n_clusters=10, random_state=0).fit(df)
    return kmeans.labels_

In [7]:
cluster_feat['clus']=do_kmeans(data_mat.iloc[:,1:])

do_kmeansは32.10807418823242秒かかりました


In [8]:
from sklearn.decomposition import PCA
@stop_watch
def do_kmeans_via_pca(df,dim):
    pca = PCA(dim)
    df_pca=pca.fit_transform(df)
    kmeans_pca = KMeans(n_clusters=10, random_state=0).fit(df_pca)
    return kmeans_pca.labels_

In [9]:
dimarr=[5,50,100,500,1000]
for dim in dimarr:
    cluster_feat[f'clus_pca{dim}']=do_kmeans_via_pca(data_mat.iloc[:,1:],dim)

do_kmeans_via_pcaは2.090344190597534秒かかりました
do_kmeans_via_pcaは2.5788679122924805秒かかりました
do_kmeans_via_pcaは4.060148000717163秒かかりました
do_kmeans_via_pcaは10.052565574645996秒かかりました
do_kmeans_via_pcaは18.522069215774536秒かかりました


In [10]:
def calc_sse(df,cluscol,clusnum):
    usecols=['gender_F', 'gender_M', 'age_1', 'age_18', 'age_25','age_35', 'age_45', 'age_50', 'age_56']
    clusdf=df[df[cluscol]==clusnum]
    meanvalue=clusdf[usecols].mean().values
    sse=0
    for row in range(clusdf.shape[0]):
        sse+=((clusdf.iloc[row,:][usecols]-meanvalue)**2).sum()
    return sse#/clusdf.shape[0]

In [11]:
cluslist=['clus', 'clus_pca5', 'clus_pca50', 'clus_pca100', 'clus_pca500', 'clus_pca1000']
for clus in cluslist:
    totalsse=0
    for i in range(10):
        totalsse+=calc_sse(cluster_feat,clus,i)
    print(clus,totalsse)

clus 7002.141532744949
clus_pca5 6978.64803102557
clus_pca50 6986.260983703498
clus_pca100 7020.58017422873
clus_pca500 6972.437429865451
clus_pca1000 7003.297341273083
